In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from bs4 import BeautifulSoup
import cloudscraper
import json

In [2]:
def crawl_nhatot_links(max_pages=10, output_file='nhatot_links.json'):
    import time
    all_links = []
    scraper = cloudscraper.create_scraper()
    for i in range(1, max_pages+1):
        print(f"Đang crawl page {i}...")
        URL = f"https://www.nhatot.com/mua-ban-nha-dat-ha-noi?page={i}"
        success = False
        for attempt in range(3):  # Thử lại tối đa 3 lần cho mỗi trang
            try:
                response = scraper.get(URL, timeout=20) # Để timeout để tránh treo
                soup = BeautifulSoup(response.text, 'html.parser')
                matching_divs = soup.find_all('div', attrs={
                    'style': 'position:relative',
                    'role': 'button',
                    'tabindex': '0'
                })
                for div in matching_divs:
                    a_tag = div.find('a', href=True)
                    link = a_tag['href'] if a_tag else None
                    h3_tag = div.find('h3')
                    title = h3_tag.get_text(strip=True) if h3_tag else None
                    if link:
                        if link.startswith('/'):
                            link = 'https://www.nhatot.com' + link # Tạo link đầy đủ
                        all_links.append({'url': link, 'title': title})
                success = True
                break  # Nếu thành công thì thoát retry
            except Exception as e:
                print(f"Lỗi page {i} lần {attempt+1}: {e}")
                time.sleep(3)
        if not success:
            print(f"Bỏ qua page {i} do lỗi liên tục.")
        # time.sleep(1)  # Nghỉ giữa các page, chậm tí cho server nó thở
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_links, f, ensure_ascii=False, indent=2)
    print(f"Đã lưu {len(all_links)} link vào {output_file}")


In [17]:

crawl_nhatot_links(max_pages=10, output_file='data/nhatot_links_test.json')

Đang crawl page 1...
Đang crawl page 2...
Đang crawl page 3...
Đang crawl page 4...
Đang crawl page 5...
Đang crawl page 6...
Đang crawl page 7...
Đang crawl page 8...
Đang crawl page 9...
Đang crawl page 10...
Đã lưu 162 link vào data/nhatot_links_test.json


In [2]:
def crawl_nhatot_details(url):
    scraper = cloudscraper.create_scraper()
    try:
        response = scraper.get(url, timeout=20)
        soup = BeautifulSoup(response.text, 'html.parser')
        divs = soup.find_all('div', class_='a1b2hkis', limit=2)
        result = {}
        for idx, div in enumerate(divs):
            if idx == 0:
                h1 = div.find('h1')
                if h1:
                    result['Tiêu đề'] = h1.get_text(strip=True)
                address_block = div.find('span', class_='bwq0cbs flex-1')
                if address_block:
                    result['Địa chỉ'] = address_block.get_text(strip=True)
                date_block = div.find('span', class_='re__card-published-info-published-at')
                if date_block:
                    result['Ngày cập nhật'] = date_block.get_text(strip=True)
            if idx == 1:
                params = div.find_all('div', class_='abzctes', attrs={'data-testid': 'param-item'})
                for param in params:
                    label = param.find('span')
                    value = param.find('strong')
                    if label and value:
                        label_text = label.get_text(strip=True)
                        value_text = value.get_text(strip=True)
                        result[label_text] = value_text
                desc = div.find('p', itemprop='description')
                if desc:
                    result['Mô tả chi tiết'] = desc.get_text(strip=True)
                phone_block = div.find('div', class_='InlineShowPhoneButton_linkContact__U_lEr')
                if phone_block:
                    result['SĐT'] = phone_block.get_text(strip=True)
        return result
    except Exception as e:
        print(f"Lỗi khi crawl {url}: {e}")
        return None

In [4]:
crawl_nhatot_details("https://www.nhatot.com/mua-ban-nha-dat-quan-ha-dong-ha-noi/125324875.htm")


{'Tiêu đề': '💥 CỰC SỐC NHÀ  70m x 5 Tầng x MẶT TIỀN 6m PHỐ LỤA GIÁ CHỈ 1x.x TỶ 💥',
 'Địa chỉ': 'Phố Lụa, Phường Vạn Phúc, Quận Hà Đông, Hà Nội',
 'Loại hình nhà ở': 'Nhà mặt phố, mặt tiền',
 'Diện tích đất': '70.3 m²',
 'Diện tích sử dụng': '70.3 m²',
 'Giá/m²': '244,67 triệu/m²',
 'Giấy tờ pháp lý': 'Đã có sổ',
 'Số phòng ngủ': '3 phòng'}

In [ ]:
import time
results = []
with open('data/nhatot_links.json', 'r', encoding='utf-8') as f:
    links = json.load(f)
for item in links[::-1]:
    url = item['url']
    title = item.get('title')
    detail = None
    for attempt in range(3):
        detail = crawl_nhatot_details(url)
        if detail and len(detail.keys()) >= 3:
            break
        time.sleep(random.uniform(1, 5))
    if detail:
        detail['url'] = url
        detail['title'] = title
        print(f"Đã crawl: {url}\n{detail}\n")
        results.append(detail)
        # Lưu ngay sau mỗi lần crawl thành công
        with open('data/nhatot_details_2.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
    else:
        print(f"Không crawl được: {url}")
print(f"Đã lưu {len(results)} kết quả vào data/nhatot_details_2.json")

Đã crawl: https://www.nhatot.com/mua-ban-nha-dat-quan-bac-tu-liem-ha-noi/125025774.htm
{'Tiêu đề': 'Lô góc, nhà mới, ngõ ô tô, full đồ ở ngay : 32m2 * 4 tầng.', 'Địa chỉ': 'phố văn hội, Phường Đức Thắng, Quận Bắc Từ Liêm, Hà Nội', 'Loại hình nhà ở': 'Nhà ngõ, hẻm', 'Diện tích đất': '32 m²', 'Giá/m²': '217,19 triệu/m²', 'Giấy tờ pháp lý': 'Đã có sổ', 'Số phòng ngủ': '4 phòng', 'Số phòng vệ sinh': '4 phòng', 'url': 'https://www.nhatot.com/mua-ban-nha-dat-quan-bac-tu-liem-ha-noi/125025774.htm', 'title': 'Lô góc, nhà mới, ngõ ô tô, full đồ ở ngay : 32m2 * 4 tầng.'}

Đã crawl: https://www.nhatot.com/mua-ban-nha-dat-quan-hoang-mai-ha-noi/125025820.htm
{'Tiêu đề': 'Ô tô tránh, Ngũ Nhạc, kinh doanh sầm uất 50m2 * 4 tầng, MT3.5 nhỉnh 7', 'Địa chỉ': 'Ngõ 130 Phố Ngũ Nhạc, Phường Thanh Trì, Quận Hoàng Mai, Hà Nội', 'Loại hình nhà ở': 'Nhà ngõ, hẻm', 'Diện tích đất': '50 m²', 'Giá/m²': '158 triệu/m²', 'Giấy tờ pháp lý': 'Đã có sổ', 'Số phòng ngủ': '3 phòng', 'url': 'https://www.nhatot.com/mua-ban-